

# Cut Frames, Give Original Labels and Merge Labels

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import os
import sys
os.chdir('/content/gdrive/My Drive/e6691-2022spring-assign2-VCSZ-yc3998-fs2752-cz2678-main/.')

In [3]:
# imports
import pickle
import torch
from torch.utils.data import Dataset, DataLoader, random_split, SequentialSampler, RandomSampler, BatchSampler
from torchvision import models, transforms
from torch import nn, optim

from PIL import Image
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from utils.clean_labels import clean_labels
from utils.mydataset import SVRCDataset

import os
from datetime import datetime
import random
import time

%load_ext autoreload
%autoreload 2

## Data Preprocessing

### Extract frames

In [8]:
# put videos here!
#video_base = 'D:/e6691/6691_assignment2/videos'
video_base = 'Surgery_Videos_Hernia'
videos = [v for v in os.listdir(video_base) if v.endswith('mp4')]
# images will be output to here'
print(videos)
#image_base = 'D:/e6691/6691_assignment2/images'
image_base = 'Surgery_Images_Hernia'
if not os.path.exists(image_base):
    os.mkdir(image_base)

['RALIHR_surgeon01_fps01_0040.mp4', 'RALIHR_surgeon01_fps01_0023.mp4', 'RALIHR_surgeon01_fps01_0070.mp4', 'RALIHR_surgeon01_fps01_0015.mp4', 'RALIHR_surgeon01_fps01_0066.mp4', 'RALIHR_surgeon01_fps01_0072.mp4', 'RALIHR_surgeon01_fps01_0083.mp4', 'RALIHR_surgeon01_fps01_0003.mp4', 'RALIHR_surgeon01_fps01_0002.mp4', 'RALIHR_surgeon01_fps01_0080.mp4', 'RALIHR_surgeon01_fps01_0073.mp4', 'RALIHR_surgeon01_fps01_0067.mp4', 'RALIHR_surgeon01_fps01_0068.mp4', 'RALIHR_surgeon01_fps01_0051.mp4', 'RALIHR_surgeon01_fps01_0078.mp4', 'RALIHR_surgeon01_fps01_0088.mp4', 'RALIHR_surgeon01_fps01_0019.mp4', 'RALIHR_surgeon01_fps01_0053.mp4', 'RALIHR_surgeon01_fps01_0049.mp4', 'RALIHR_surgeon01_fps01_0030.mp4', 'RALIHR_surgeon01_fps01_0034.mp4', 'RALIHR_surgeon01_fps01_0075.mp4', 'RALIHR_surgeon01_fps01_0054.mp4', 'RALIHR_surgeon01_fps01_0033.mp4', 'RALIHR_surgeon01_fps01_0064.mp4', 'RALIHR_surgeon01_fps01_0055.mp4', 'RALIHR_surgeon01_fps01_0039.mp4', 'RALIHR_surgeon01_fps01_0086.mp4', 'RALIHR_surgeon01_f

In [ ]:
# command line: 
# ffmpeg -i {input_video} -r {frame_rate} [-f {force_format} (not needed)] {output_images}
# doc: https://ffmpeg.org/ffmpeg.html
videos = sorted(videos)
for video in videos:
    input_path = os.path.join(video_base, video)
    # make dirs
    output_base = image_base + '/{}'.format(video.split('.')[0])
    if not os.path.exists(output_base):
        os.mkdir(output_base)
    output_path = os.path.join(output_base, '%d.png')
    # command
    print('Frames extracted from {} to {}'.format(input_path, output_path))
    !ffmpeg -i {input_path} -r 1 {output_path}

### Extract labels

In [9]:
# some useless rows are manually deleted
# labels_path = 'D:/e6691/6691_assignment2'
labels_path = 'data/labels/video.phase.train_test.clean.csv'
# names_path = 'D:/e6691/6691_assignment2'
names_path = 'data/labels/all_labels_hernia.csv'

# this is already a cleaned file
labels_df_cleaned = pd.read_csv(labels_path)
names_df = pd.read_csv(names_path, names=['Name'])

In [11]:
names_df

,Name
id,labels
0,adhesiolysis
1,blurry
2,catheter insertion
3,mesh placement
4,mesh positioning
5,out of body
6,peritoneal closure
7,peritoneal scoring
8,positioning suture


In [44]:
labels_df_cleaned

,VideoName,PhaseName,Start,End
0,RALIHR_surgeon01_fps01_0001,stationary idle,00:00,00:16
1,RALIHR_surgeon01_fps01_0001,transitionary idle,00:16,00:35
2,RALIHR_surgeon01_fps01_0001,out of body,00:35,01:05
3,RALIHR_surgeon01_fps01_0001,transitionary idle,01:05,01:59
4,RALIHR_surgeon01_fps01_0001,peritoneal scoring,01:59,02:55
...,...,...,...,...
5575,RALIHR_surgeon01_fps01_0188,catheter insertion,22:26,23:22
5576,RALIHR_surgeon01_fps01_0188,transitionary idle,23:22,23:28
5577,RALIHR_surgeon01_fps01_0188,positioning suture,23:28,23:45
5578,RALIHR_surgeon01_fps01_0188,peritoneal closure,23:45,30:41


In [48]:
# convert time string to int
# start time
t0 = datetime(1900, 1, 1)
def time2int(t):
    # if convertable
    return (
        # time given in MM:SS
        datetime.strptime(t, '%M:%S') - t0
    ).seconds if len(t.split(':')) == 2 else (
        # time given in HH:MM:SS
        datetime.strptime(t, '%H:%M:%S') - t0
    ).seconds

# extract names
for video in sorted(videos)[103:189]:
    print('now processing ' + video)
    images = os.listdir(os.path.join(image_base, video.split('.')[0]))
    # get df corresponding to current video
    video_df = labels_df_cleaned.loc[labels_df_cleaned['VideoName'] == video.split('.')[0]]
    # add two columns
    video_df[['StartSec', 'EndSec']] = video_df[['Start', 'End']].applymap(time2int)
    # print(video_df)
    # 这warning好烦但我懒得改了
    for image in images:
        # print(image)
        # check paths
        base = os.path.join(image_base, video.split('.')[0])
        path = os.path.join(base, image)
        # if '-' in image:
        #     continue
        t = int(image.split('.')[0].split('-')[0])
        # select interval and remove tailing digits
        name = video_df[
            (video_df['StartSec'] <= t) & (t <= video_df['EndSec'])
        ]['PhaseName'].iloc[0]
        # find correct integer labels
        label = names_df[names_df['Name'] == name].index[0]
        # incorperate label into filenames
        if '-' not in path:
            # avoid renaming twice
            #new_path = ''.join(path.split('.')[:-1]) + '-{}.'.format(label) + path.split('.')[-1]
            new_path = '{}-{}.{}'.format(''.join(path.split('.')[:-1]), label, path.split('.')[-1])
        else:
            #new_path = path.split('-')[0] + '-' + str(label) + '.' + path.split('-')[1].split('.')[1]
            new_path = '{}-{}.{}'.format(path.split('-')[0], label, path.split('-')[1].split('.')[1])
        # rename all files
        os.rename(path, new_path)

now processing RALIHR_surgeon01_fps01_0104.mp4


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


now processing RALIHR_surgeon01_fps01_0105.mp4
now processing RALIHR_surgeon01_fps01_0106.mp4
now processing RALIHR_surgeon01_fps01_0107.mp4
now processing RALIHR_surgeon01_fps01_0108.mp4
now processing RALIHR_surgeon01_fps01_0109.mp4
now processing RALIHR_surgeon01_fps01_0110.mp4
now processing RALIHR_surgeon01_fps01_0111.mp4
now processing RALIHR_surgeon01_fps01_0112.mp4
now processing RALIHR_surgeon01_fps01_0113.mp4
now processing RALIHR_surgeon01_fps01_0114.mp4
now processing RALIHR_surgeon01_fps01_0115.mp4
now processing RALIHR_surgeon01_fps01_0116.mp4
now processing RALIHR_surgeon01_fps01_0117.mp4
now processing RALIHR_surgeon01_fps01_0118.mp4
now processing RALIHR_surgeon01_fps01_0119.mp4
now processing RALIHR_surgeon01_fps01_0120.mp4
now processing RALIHR_surgeon01_fps01_0121.mp4
now processing RALIHR_surgeon01_fps01_0122.mp4
now processing RALIHR_surgeon01_fps01_0123.mp4
now processing RALIHR_surgeon01_fps01_0124.mp4
now processing RALIHR_surgeon01_fps01_0125.mp4
now processin

IndexError: ignored

## Dataset Construction

In [6]:
# Data properties
num_labels = 14

In [10]:
# get all images and labels
image_paths = []
labels = []

for video in sorted(videos)[:188]:
    base = os.path.join(image_base, video.split('.')[0])
    image_paths.append(list(map(
        lambda img: base + '/' + img if img.endswith('.png') else None, 
        os.listdir(base)
    )))


image_paths = [list(filter(None, images)) for images in image_paths]
#image_paths = [list(sorted(images, key=sort_images)) for images in image_paths]


In [11]:
# Because the 103rd video has some problems about labels, so we skip the 103rd video 
image_paths.pop(102)

['Surgery_Images_Hernia/RALIHR_surgeon01_fps01_0103/843-10.png',
 'Surgery_Images_Hernia/RALIHR_surgeon01_fps01_0103/844-10.png',
 'Surgery_Images_Hernia/RALIHR_surgeon01_fps01_0103/845-10.png',
 'Surgery_Images_Hernia/RALIHR_surgeon01_fps01_0103/846-10.png',
 'Surgery_Images_Hernia/RALIHR_surgeon01_fps01_0103/847-10.png',
 'Surgery_Images_Hernia/RALIHR_surgeon01_fps01_0103/848-10.png',
 'Surgery_Images_Hernia/RALIHR_surgeon01_fps01_0103/849-10.png',
 'Surgery_Images_Hernia/RALIHR_surgeon01_fps01_0103/850-10.png',
 'Surgery_Images_Hernia/RALIHR_surgeon01_fps01_0103/851-10.png',
 'Surgery_Images_Hernia/RALIHR_surgeon01_fps01_0103/852-10.png',
 'Surgery_Images_Hernia/RALIHR_surgeon01_fps01_0103/853-10.png',
 'Surgery_Images_Hernia/RALIHR_surgeon01_fps01_0103/854-10.png',
 'Surgery_Images_Hernia/RALIHR_surgeon01_fps01_0103/855-10.png',
 'Surgery_Images_Hernia/RALIHR_surgeon01_fps01_0103/856-10.png',
 'Surgery_Images_Hernia/RALIHR_surgeon01_fps01_0103/857-10.png',
 'Surgery_Images_Hernia/R

In [12]:
labels = []

label_before = 2

for images in image_paths:
  temp = []
  for img in images:
    
    label = int(img.split('.')[0].split('-')[1]) # get the original label

    # merge the small amount labels:
    # Adhesiolysis -> peritoneal dissection (0 -> 9)
    # Blurry -> which to merge with: adjacent phases in the video (1 -> before)
    # Mesh positioning -> mesh placement (4 -> 3)
    # Primary hernia repair -> reduction (10 -> 11)
    # Stationary idle -> transitionary idle (12 -> 13)

    if label == 0:
      label = 9
    elif label == 1:
      label = label_before
    elif label == 4:
      label = 3
    elif label == 10:
      label = 11
    elif label == 12:
      label = 13
    label_before = label
    temp.append(label)
  labels.append(temp)



# labels = [
#     [int(img.split('.')[0].split('-')[1]) for img in images]
#     for images in image_paths
# ]


In [13]:
labels_merged = list(zip(image_paths, labels))

In [14]:
len(labels_merged)

187

In [17]:
filehandler = open(b"labels_merged.pkl","wb")
pickle.dump(labels_merged,filehandler)